In [1]:
import timm
import torch
import zipfile, os
from PIL import Image
from pathlib import Path
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

c:\ProgramData\anaconda3\envs\ultralytics_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Check CUDA availability
print(torch.cuda.is_available())  # Should return True
print(torch.version.cuda)         # Should match the installed CUDA version

True
11.8


In [3]:
local_zip = os.path.join(os.getcwd(), 'GCD.zip')
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(os.getcwd())
zip_ref.close()

train_dir = os.path.join(os.getcwd(), 'GCD', 'train')
test_dir = os.path.join(os.getcwd(), 'GCD', 'test')

In [4]:
# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

In [5]:
# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_names = os.listdir(data_dir)

        for label, class_name in enumerate(self.class_names):
            class_dir = os.path.join(data_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.images.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
# Data loaders
train_dataset = CustomDataset(data_dir=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
# Function for training and evaluation
def train_and_evaluate_model(model_name, model, train_loader, test_loader, num_epochs=10):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print(f"Training model: {model_name}")

    # Training loop
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        epoch_loss = 0  # To accumulate loss for the epoch
        correct_predictions = 0
        total_samples = 0
        all_targets = []
        all_preds = []

        progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")  # Progress bar

        for inputs, targets in progress_bar:
            inputs, targets = inputs.to(device), targets.to(device)  # Move to device

            optimizer.zero_grad()  # Zero the gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, targets)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            # Update progress bar description with the current loss
            epoch_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

            # Calculate accuracy
            _, preds = torch.max(outputs, dim=1)
            correct_predictions += (preds == targets).sum().item()
            total_samples += targets.size(0)
            all_targets.extend(targets.tolist())
            all_preds.extend(preds.tolist())

        avg_loss = epoch_loss / len(train_loader)
        accuracy = correct_predictions / total_samples * 100

        # Calculate precision, recall, and F1-score for the train set
        precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted')
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {avg_loss:.4f}, "
              f"Train Accuracy: {accuracy:.2f}%, "
              f"Train Precision: {precision:.2f}, "
              f"Train Recall: {recall:.2f}, "
              f"Train F1-Score: {f1:.2f}")

        # Evaluate the model on the test set after each epoch
        model.eval()  # Set model to evaluation mode
        correct_predictions = 0
        total_samples = 0
        all_targets = []
        all_preds = []

        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, dim=1)
                correct_predictions += (preds == targets).sum().item()
                total_samples += targets.size(0)
                all_targets.extend(targets.tolist())
                all_preds.extend(preds.tolist())

        test_accuracy = correct_predictions / total_samples * 100

        # Calculate precision, recall, and F1-score for the test set
        precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted')
        print(f"Test Accuracy for {model_name} after epoch {epoch+1}: {test_accuracy:.2f}%")
        print(f"Test Precision: {precision:.2f}, Test Recall: {recall:.2f}, Test F1-Score: {f1:.2f}\n")

In [10]:
# Models to train
models = [
    ('maxvit_tiny_tf_224.in1k', timm.create_model('maxvit_tiny_tf_224.in1k', pretrained=True, num_classes=7)),
    ('maxvit_tiny_rw_224.sw_in1k', timm.create_model('maxvit_tiny_rw_224.sw_in1k', pretrained=True, num_classes=7)),
    ('maxvit_small_tf_224.in1k', timm.create_model('maxvit_small_tf_224.in1k', pretrained=True, num_classes=7))
]

c:\ProgramData\anaconda3\envs\ultralytics_env\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--timm--maxvit_tiny_rw_224.sw_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:
# Train and evaluate each model
for model_name, model in models:
    train_and_evaluate_model(model_name, model, train_loader, test_loader, num_epochs=10)

Training model: maxvit_tiny_tf_224.in1k


Epoch [1/10]: 100%|██████████| 313/313 [43:56<00:00,  8.42s/it, loss=0.562]


Epoch [1/10], Train Loss: 0.5747, Train Accuracy: 78.68%, Train Precision: 0.78, Train Recall: 0.79, Train F1-Score: 0.78
Test Accuracy for maxvit_tiny_tf_224.in1k after epoch 1: 67.81%
Test Precision: 0.76, Test Recall: 0.68, Test F1-Score: 0.67



Epoch [2/10]: 100%|██████████| 313/313 [43:38<00:00,  8.36s/it, loss=0.352] 


Epoch [2/10], Train Loss: 0.3970, Train Accuracy: 85.46%, Train Precision: 0.85, Train Recall: 0.85, Train F1-Score: 0.85
Test Accuracy for maxvit_tiny_tf_224.in1k after epoch 2: 73.62%
Test Precision: 0.76, Test Recall: 0.74, Test F1-Score: 0.73



...

Epoch [9/10], Train Loss: 0.2654, Train Accuracy: 89.71%, Train Precision: 0.90, Train Recall: 0.90, Train F1-Score: 0.90
Test Accuracy for maxvit_tiny_rw_224.sw_in1k after epoch 9: 73.68%
Test Precision: 0.78, Test Recall: 0.74, Test F1-Score: 0.73



Epoch [10/10]: 100%|██████████| 313/313 [43:04<00:00,  8.26s/it, loss=0.101] 


Epoch [10/10], Train Loss: 0.2579, Train Accuracy: 90.25%, Train Precision: 0.90, Train Recall: 0.90, Train F1-Score: 0.90
Test Accuracy for maxvit_tiny_rw_224.sw_in1k after epoch 10: 72.32%
Test Precision: 0.76, Test Recall: 0.72, Test F1-Score: 0.72

Training model: maxvit_small_tf_224.in1k


Epoch [1/10]:   0%|          | 0/313 [00:04<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 158.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 10.54 GiB is allocated by PyTorch, and 147.12 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)